In [ ]:
import os
import shutil
import pandas as pd
from PIL import Image
import numpy as np
#import matplotlib.pyplot as plt
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import cv2

img_rows = 150
img_cols = 224
color_type = 3
batch_size=48
epochs=300
subject='Severstal'
main_path=os.path.join("E:\\kaggle_imgs",subject)
img_path=os.path.join(main_path,"images")
data_path=os.path.join(main_path,"Data")
saved_path=os.path.join(main_path,"saved_models")
paths=[main_path, img_path,saved_path,data_path]
for fp in paths:
        print(fp)
        if not os.path.exists(fp):        
                os.mkdir(fp)
file_path=os.path.join(saved_path,subject+"200608_")
file_best=os.path.join(saved_path,subject+"200608_")

train_img_pkl=os.path.join(data_path,"train_imgs.npy")
test_img_pkl=os.path.join(data_path,"test_imgs.npy")
train_info_pkl=os.path.join(data_path,"df_train_pickle.csv")

num_classes=4

## Configure parameters

In [ ]:
from tqdm import tqdm_notebook
import os
def get_train_info():
        df=pd.read_csv(os.path.join(data_path,"train.csv"))
        files = os.listdir(os.path.join(data_path, "train_images"))
        df_all_images=pd.DataFrame({"ImageId":files})
        df_NoDefect=df_all_images[~df_all_images.ImageId.isin(df.ImageId)]
        df_NoDefect["ClassId"]=0
        df_NoDefect["EncodedPixels"]=np.NaN
        df_train=pd.concat([df,df_NoDefect]).reset_index(drop=True)    
    
        legacy_df=pd.DataFrame(columns=["ImageId_ClassId","EncodedPixels"])
        my_group=df_train.groupby("ImageId")
        for img_id,img_df in tqdm_notebook(my_group):
                for i in range(1,5):
                        avail_classes = list(img_df.ClassId)
                        row = dict()
                        row['ImageId_ClassId'] = img_id + '_' + str(i)

                        if i in avail_classes:
                                row['EncodedPixels'] = img_df.loc[img_df.ClassId == i].EncodedPixels.iloc[0]
                        else:
                                row['EncodedPixels'] = np.nan

                        legacy_df = legacy_df.append(row, ignore_index=True)
        return legacy_df

In [ ]:
fp=train_info_pkl
if os.path.exists(fp):
    df_train=pd.read_csv(fp)
    print("df_train load complete")
else:
    df_train=get_train_info()
    df_train['ImageId'] = df_train['ImageId_ClassId'].map(lambda x: x.split('_')[0])
    df_train['ClassId'] = df_train['ImageId_ClassId'].map(lambda x: x.split('_')[1])
    df_train['HavingDefection'] = df_train['EncodedPixels'].map(lambda x: 0 if x is np.nan else 1)
    df_train.to_csv(fp,index=False)

## visualize mask

In [ ]:
palet = [(249, 192, 12), (0, 185, 241), (114, 0, 218), (249,50,12)]

In [ ]:
def name_and_mask(start_idx):
    col = start_idx
    img_names = [str(i).split("_")[0] for i in df_train.iloc[col:col+4, 0].values]
#     if not (img_names[0] == img_names[1] == img_names[2] == img_names[3]):
#         raise ValueError

    labels = df_train.iloc[col:col+4, 1]
    mask = np.zeros((256, 1600, 4), dtype=np.uint8)

    for idx, label in enumerate(labels.values):
        if label is not np.nan:
            mask_label = np.zeros(1600*256, dtype=np.uint8)
            label = label.split(" ")
            positions = map(int, label[0::2])
            length = map(int, label[1::2])
            for pos, le in zip(positions, length):
                mask_label[pos-1:pos+le-1] = 1
            mask[:, :, idx] = mask_label.reshape(256, 1600, order='F')
    return img_names[0], mask

In [ ]:
def show_mask_image(col):
    name, mask = name_and_mask(col)
    img = cv2.imread(os.path.join(data_path, f"train_images/{name}"))
    fig, ax = plt.subplots(figsize=(15, 15))

    for ch in range(4):
        _,contours,_ = cv2.findContours(mask[:, :, ch].astype(np.uint8), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        for i in range(0, len(contours)):
            cv2.polylines(img, contours[i], True, palet[ch], 2)
    ax.set_title(name)
    ax.imshow(img)
    plt.show()

In [ ]:
for i in range(30):
    if df_train.iloc[i,2]==1:
                     show_mask_image(i)

### to simplify , n this kernel i use only images with classid 4

In [ ]:
tr=df_train.copy()
df_train = tr[tr['EncodedPixels'].notnull()].reset_index(drop=True)
df_train = df_train[df_train['ImageId_ClassId'].apply(lambda x: x.split('_')[1] == '4')].reset_index(drop=True)
print(len(df_train))
df_train.head()

### decode mask

In [ ]:
def rle2mask(rle, imgshape):
    width = imgshape[0]
    height= imgshape[1]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        current_position += lengths[index]
        
    return np.flipud( np.rot90( mask.reshape(height, width), k=1 ) )

## Display some images

In [ ]:
""" MY Test"""
img = cv2.imread( data_path + '/train_images/000f6bf48.jpg' )
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
mask = rle2mask(df_train['EncodedPixels'].iloc[i], (256, 1600))
img[mask==1,0] = 64
plt.imshow(mask)
print("mask",mask.sum(),mask.shape)
print(img.shape)

In [ ]:
columns = 1
rows = 4

for i in range(30):
    if df_train.iloc[i,2]==0:
        continue
    fig = plt.figure(figsize=(20,10))
    fn = df_train.iloc[i,3]
    #fig.add_subplot(rows, columns, i).set_title(fn)
    img = cv2.imread( data_path + '/train_images/'+fn )
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = rle2mask(df_train['EncodedPixels'].iloc[i], (256, 1600))
    img[mask==1,0] = 255
    plt.imshow(img)
plt.show()

## Create train Dataset ans DataLoader

In [ ]:
from torch.utils.data import DataLoader, Dataset
from albumentations import (Normalize, Compose)
from albumentations.torch import ToTensor
from torchvision import transforms

class ImageData(Dataset):
    def __init__(self, df, transform, subset="train"):
        super().__init__()
        self.df = df
        self.transform = transform
        self.subset = subset
        
        if self.subset == "train":
            self.data_path = data_path + '/train_images/'
        elif self.subset == "test":
            self.data_path = data_path + '/test_images/'

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):                      
        fn = self.df['ImageId_ClassId'].iloc[index].split('_')[0]         
        img = Image.open(self.data_path + fn)
        img = self.transform(img)

        if self.subset == 'train': 
            mask = rle2mask(self.df['EncodedPixels'].iloc[index], (256, 1600))
            mask = transforms.ToPILImage()(mask)            
            mask = self.transform(mask)
            return img, mask
        else: 
            mask = None
            return img   


In [ ]:
data_transf = transforms.Compose([
                                  transforms.Scale((256, 256)),
                                  transforms.ToTensor()])
train_data = ImageData(df = df_train, transform = data_transf)
train_loader = DataLoader(dataset = train_data, batch_size=4)

In [ ]:
for img,mask in train_loader:
    break

In [ ]:
plt.imshow(img[1].permute(1,2,0))

In [ ]:
plt.imshow(train_data[5][0].permute(1, 2, 0))

In [ ]:
plt.imshow(np.squeeze(train_data[5][1].permute(1, 2, 0)))

## Create Unet Model

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
def convrelu(in_channels, out_channels, kernel, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel, padding=padding),
        nn.ReLU(inplace=True),
    )

class UNet(nn.Module):
    def __init__(self, n_class):
        super().__init__()
        
        self.base_model = models.resnet18()
        #self.base_model.load_state_dict(torch.load("../input/resnet18/resnet18.pth"))
        self.base_layers = list(self.base_model.children())

        self.layer0 = nn.Sequential(*self.base_layers[:3])
        self.layer0_1x1 = convrelu(64, 64, 1, 0)
        self.layer1 = nn.Sequential(*self.base_layers[3:5])
        self.layer1_1x1 = convrelu(64, 64, 1, 0)
        self.layer2 = self.base_layers[5]
        self.layer2_1x1 = convrelu(128, 128, 1, 0)
        self.layer3 = self.base_layers[6]
        self.layer3_1x1 = convrelu(256, 256, 1, 0)
        self.layer4 = self.base_layers[7]
        self.layer4_1x1 = convrelu(512, 512, 1, 0)

        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv_up3 = convrelu(256 + 512, 512, 3, 1)
        self.conv_up2 = convrelu(128 + 512, 256, 3, 1)
        self.conv_up1 = convrelu(64 + 256, 256, 3, 1)
        self.conv_up0 = convrelu(64 + 256, 128, 3, 1)

        self.conv_original_size0 = convrelu(3, 64, 3, 1)
        self.conv_original_size1 = convrelu(64, 64, 3, 1)
        self.conv_original_size2 = convrelu(64 + 128, 64, 3, 1)

        self.conv_last = nn.Conv2d(64, n_class, 1)

    def forward(self, input):
        x_original = self.conv_original_size0(input)
        x_original = self.conv_original_size1(x_original)

        layer0 = self.layer0(input)
        layer1 = self.layer1(layer0)
        layer2 = self.layer2(layer1)
        layer3 = self.layer3(layer2)
        layer4 = self.layer4(layer3)

        layer4 = self.layer4_1x1(layer4)
        x = self.upsample(layer4)
        layer3 = self.layer3_1x1(layer3)
        x = torch.cat([x, layer3], dim=1)
        x = self.conv_up3(x)

        x = self.upsample(x)
        layer2 = self.layer2_1x1(layer2)
        x = torch.cat([x, layer2], dim=1)
        x = self.conv_up2(x)

        x = self.upsample(x)
        layer1 = self.layer1_1x1(layer1)
        x = torch.cat([x, layer1], dim=1)
        x = self.conv_up1(x)

        x = self.upsample(x)
        layer0 = self.layer0_1x1(layer0)
        x = torch.cat([x, layer0], dim=1)
        x = self.conv_up0(x)

        x = self.upsample(x)
        x = torch.cat([x, x_original], dim=1)
        x = self.conv_original_size2(x)

        out = self.conv_last(x)

        return out

In [ ]:
model=UNet(n_class=1).cuda()
criterion=nn.BCEWithLogitsLoss()
optimizer=torch.optim.SGD(model.parameters(),weight_decay=1e-4,lr=0.001,momentum=0.9)


## training

In [ ]:
%%time
for epoch in range(50):
    model.train()
    for i,(data,target) in enumerate(train_loader):
        data,target=data.cuda(),target.cuda()
        optimizer.zero_grad()
        output=model(data)
        loss=criterion(output,target)
        loss.backward()
        optimizer.step()
    print(f"epoch {epoch}, Loss {loss}")

## show prediction on image from train dataset

In [ ]:
plt.imshow(train_data[6][0].permute(1, 2, 0))

In [ ]:
x = train_data[6][0].unsqueeze(0)
o = model(x.cuda())  
o = o.cpu().detach().numpy() * (-1)
tmp = np.copy(o)
mn = np.mean(o)*1.2
tmp[tmp<mn] = 0
tmp[tmp>mn] = 1
plt.imshow(np.squeeze(tmp))

## Read submit file

In [ ]:
submit=pd.read_csv(data_path+"/sample_submission.csv")

## Create test Dataset and DataLoader

In [ ]:
test_data=ImageData(df=sub4)

In [ ]:
submit.columns

In [ ]:
df_train.columns